In [20]:
# --- PASO 1: Instalación de librerías necesarias
!pip install --upgrade gspread pandas gspread-dataframe scikit-learn tqdm ipywidgets tensorflow matplotlib seaborn

# --- PASO 2: Importación de librerías
from google.colab import auth
auth.authenticate_user()

import random
import gspread
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from sklearn.utils import class_weight
from gspread_dataframe import set_with_dataframe
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from google.colab import output
output.enable_custom_widget_manager()
from google.auth import default
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter

random.seed(42)
np.random.seed(42)

# --- PASO 3: Conexión a Google Sheets
spreadsheet_id = '18J8OHNlZRBNWMfA_hHnltZsrM2vRi8qji3dr20hAPEw'
link_google_sheets = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit?usp=sharing"
creds, _ = default()
gc = gspread.authorize(creds)
sh = gc.open_by_key(spreadsheet_id)
worksheet = sh.sheet1

# --- PASO 4: Carga y validación de datos
values = worksheet.get_all_values()
headers = values[0]
rows = values[1:]
df = pd.DataFrame(rows, columns=headers)

# Validar columnas esperadas (1 a 98)
esperadas = [f"{i}." for i in range(1, 99)]
preguntas_validas = df.columns[3:]
for col in preguntas_validas:
    if not any(col.strip().startswith(e) for e in esperadas):
        raise ValueError(f"❌ Columna de pregunta mal etiquetada: {col}")

if df.columns.duplicated().any():
    raise ValueError("❌ Hay columnas duplicadas en el dataset.")

# Convertir respuestas a números
for col in preguntas_validas:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Validar respuestas
if df[preguntas_validas].isnull().any().any():
    raise ValueError("🚫 Existen respuestas vacías o no válidas.")
if not df[preguntas_validas].isin([0, 1]).all().all():
    raise ValueError("🚫 Las respuestas deben ser 0 o 1 únicamente.")

df[preguntas_validas] = df[preguntas_validas].astype(int)
print("✅ Datos cargados y validados correctamente.")
# --- PASO 5: Definición de mapeos CHASIDE
mapeo_interes = {
    'C': [98, 12, 64, 53, 85, 1, 78, 20, 71, 91],
    'H': [9, 34, 80, 25, 95, 67, 41, 74, 56, 89],
    'A': [21, 45, 96, 57, 28, 11, 50, 3, 81, 36],
    'S': [33, 92, 70, 8, 87, 62, 23, 44, 16, 52],
    'I': [75, 6, 19, 38, 60, 27, 83, 54, 47, 97],
    'D': [84, 31, 48, 73, 5, 65, 14, 37, 58, 24],
    'E': [77, 42, 88, 17, 93, 32, 68, 49, 35, 61]
}
mapeo_aptitud = {
    'C': [15, 51, 2, 46],
    'H': [63, 30, 72, 86],
    'A': [22, 39, 76, 82],
    'S': [69, 40, 29, 4],
    'I': [26, 59, 90, 10],
    'D': [13, 66, 18, 43],
    'E': [94, 7, 79, 55]
}
area_completa = {
    'C': 'Administrativas y Contables',
    'H': 'Humanísticas y Sociales',
    'A': 'Artísticas',
    'S': 'Medicina y Ciencias de la Salud',
    'I': 'Enseñanzas Técnicas',
    'D': 'Defensa y Seguridad',
    'E': 'Ciencias Experimentales'
}
clases = list(area_completa.keys())

# --- PASO 6: Cálculo de etiquetas
def calcular_puntajes(row, mapeo):
    puntajes = {key: 0 for key in mapeo}
    for letra, preguntas in mapeo.items():
        for pregunta in preguntas:
            col_match = [col for col in row.index if col.strip().startswith(f"{pregunta}.")]
            if col_match and row[col_match[0]] == 1:
                puntajes[letra] += 1
    return puntajes

X = df.iloc[:, 3:]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

y_interes, y_aptitud = [], []
for _, row in df.iterrows():
    puntajes_i = calcular_puntajes(row, mapeo_interes)
    puntajes_a = calcular_puntajes(row, mapeo_aptitud)
    y_interes.append(max(puntajes_i, key=puntajes_i.get))
    y_aptitud.append(max(puntajes_a, key=puntajes_a.get))

label_map = {c: i for i, c in enumerate(clases)}
y_interes_num = np.array([label_map[val] for val in y_interes])
y_aptitud_num = np.array([label_map[val] for val in y_aptitud])

# --- Validar balance por clase
def validar_balance(y_labels, nombre):
    conteo = Counter(y_labels)
    for clase, cantidad in conteo.items():
        if cantidad < 5:
            print(f"⚠ Clase con pocos datos: {clases[clase]} ({cantidad}) en {nombre}")

validar_balance(y_interes_num, "interés")
validar_balance(y_aptitud_num, "aptitud")

# --- PASO 7: División de datos y entrenamiento
X_train, X_test, y_interes_train, y_interes_test = train_test_split(X_scaled, y_interes_num, test_size=0.2, stratify=y_interes_num, random_state=42)
_, _, y_aptitud_train, y_aptitud_test = train_test_split(X_scaled, y_aptitud_num, test_size=0.2, stratify=y_aptitud_num, random_state=42)

def crear_modelo(input_dim, num_clases):
    model = Sequential([
        tf.keras.Input(shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(num_clases, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

pesos_interes = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_interes_train), y=y_interes_train)
pesos_aptitud = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_aptitud_train), y=y_aptitud_train)

modelo_interes = crear_modelo(X_train.shape[1], len(clases))
modelo_aptitud = crear_modelo(X_train.shape[1], len(clases))

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

modelo_interes.fit(X_train, y_interes_train, epochs=100, validation_split=0.2,
                   class_weight=dict(enumerate(pesos_interes)), callbacks=[early_stop], verbose=0)

modelo_aptitud.fit(X_train, y_aptitud_train, epochs=100, validation_split=0.2,
                   class_weight=dict(enumerate(pesos_aptitud)), callbacks=[early_stop], verbose=0)

print("✅ Modelos entrenados correctamente.")
# --- PASO 8: Predicciones
y_interes_pred = np.argmax(modelo_interes.predict(X_test), axis=1)
y_aptitud_pred = np.argmax(modelo_aptitud.predict(X_test), axis=1)

# --- PASO 9: Mostrar resultado en HTML
def mostrar_resultado(nombre, area_interes, area_aptitud):
    html = f"""
    <div style="background-color:#F0FFFF; padding:20px; margin-top:20px; border:1px solid #00CED1; border-radius:10px;">
        <h2 style="color:#1E90FF;">👤 {nombre}</h2>
        <h3 style="color:#20B2AA;">🏛️ Interés: {area_interes}</h3>
        <h3 style="color:#32CD32;">🎯 Aptitud: {area_aptitud}</h3>
    </div>
    """
    display(HTML(html))

# --- PASO 10: Exportar resultados a Google Sheets
def guardar_resultados(resultados):
    df_resultados = pd.DataFrame(resultados)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_hoja = f"Resultados_TF_{timestamp}"
    hoja_nueva = sh.add_worksheet(title=nombre_hoja, rows="100", cols="30")
    set_with_dataframe(hoja_nueva, df_resultados)
    print(f"✅ Resultados guardados en la hoja: {nombre_hoja}")

# --- PASO 11: Evaluar todos los estudiantes
def evaluar_todos():
    resultados = []
    columnas_entrada = X.columns  # Usar las columnas originales del entrenamiento

    for _, alumno in tqdm(df.iterrows(), total=len(df)):
        nombre = alumno['Nombre y Apellidos']
        X_input = scaler.transform(pd.DataFrame([alumno[columnas_entrada].values], columns=columnas_entrada))

        pred_int = np.argmax(modelo_interes.predict(X_input), axis=1)[0]
        pred_apt = np.argmax(modelo_aptitud.predict(X_input), axis=1)[0]
        letra_int = clases[pred_int]
        letra_apt = clases[pred_apt]

        puntajes_i = calcular_puntajes(alumno, mapeo_interes)
        puntajes_a = calcular_puntajes(alumno, mapeo_aptitud)
        puntajes = {f"{k}_int": puntajes_i[k] for k in area_completa}
        puntajes.update({f"{k}_apt": puntajes_a[k] for k in area_completa})

        resultados.append({
            'Nombre': nombre,
            **puntajes,
            'Área de Interés': area_completa[letra_int],
            'Área de Aptitud': area_completa[letra_apt],
            'Perfil CHASIDE': f"{letra_int}-{letra_apt}"
        })

    guardar_resultados(resultados)
    mostrar_botones_finales()
# --- PASO 12: Evaluar un estudiante individual
def evaluar_uno(nombre_input):
    alumno = df[df['Nombre y Apellidos'] == nombre_input]
    if alumno.empty:
        print("🚫 Alumno no encontrado.")
        return

    alumno = alumno.iloc[0]
    columnas_entrada = X.columns

    X_input = scaler.transform(pd.DataFrame([alumno[columnas_entrada].values], columns=columnas_entrada))

    pred_int = np.argmax(modelo_interes.predict(X_input), axis=1)[0]
    pred_apt = np.argmax(modelo_aptitud.predict(X_input), axis=1)[0]
    letra_int = clases[pred_int]
    letra_apt = clases[pred_apt]

    mostrar_resultado(alumno['Nombre y Apellidos'], area_completa[letra_int], area_completa[letra_apt])

    puntajes_i = calcular_puntajes(alumno, mapeo_interes)
    puntajes_a = calcular_puntajes(alumno, mapeo_aptitud)
    puntajes = {f"{k}_int": puntajes_i[k] for k in area_completa}
    puntajes.update({f"{k}_apt": puntajes_a[k] for k in area_completa})

    resultado = {
        'Nombre': alumno['Nombre y Apellidos'],
        **puntajes,
        'Área de Interés': area_completa[letra_int],
        'Área de Aptitud': area_completa[letra_apt],
        'Perfil CHASIDE': f"{letra_int}-{letra_apt}"
    }

    guardar_resultados([resultado])
    mostrar_botones_finales()

# --- PASO 13: Reportes métricos
def ver_reportes():
    from sklearn.metrics import classification_report
    import matplotlib.pyplot as plt
    import pandas as pd
    from IPython.display import clear_output, display, HTML

    clear_output()

    print("Reporte de clasificación - Interés:")
    print(classification_report(y_interes_test, y_interes_pred, target_names=clases, zero_division=0))
    acc_interes = round(accuracy_score(y_interes_test, y_interes_pred) * 100, 2)
    print("Exactitud interés:", acc_interes, "%")

    print("\nReporte de clasificación - Aptitud:")
    print(classification_report(y_aptitud_test, y_aptitud_pred, target_names=clases, zero_division=0))
    acc_aptitud = round(accuracy_score(y_aptitud_test, y_aptitud_pred) * 100, 2)
    print("Exactitud aptitud:", acc_aptitud, "%")

    # --- NUEVO: gráfico de líneas en lugar de barras
    def plot_classification_report(y_true, y_pred, nombre_modelo):
        reporte = classification_report(y_true, y_pred, target_names=clases, output_dict=True, zero_division=0)
        df_rep = pd.DataFrame(reporte).transpose()
        df_rep = df_rep.iloc[:len(clases)][['precision', 'recall', 'f1-score']]

        plt.figure(figsize=(10, 5))
        for col in df_rep.columns:
            plt.plot(df_rep.index, df_rep[col], marker='o', label=col)

        plt.title(f"Reporte de clasificación - {nombre_modelo}")
        plt.xlabel("Clase")
        plt.ylabel("Valor")
        plt.xticks(rotation=0)
        plt.ylim(0, 1)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

    # Mostrar INTERÉS
    plot_classification_report(y_interes_test, y_interes_pred, "Interés")
    display(HTML("<br><br><hr style='border:1px solid #ccc;'><br><br>"))
    # Mostrar APTITUD
    plot_classification_report(y_aptitud_test, y_aptitud_pred, "Aptitud")

    mostrar_botones_finales()

# --- FUNCIÓN auxiliar: volver al menú y mostrar botón Sheets
def mostrar_botones_finales():
    boton_volver = widgets.Button(
        description="🔄 VOLVER AL MENÚ",
        button_style='info',
        layout=widgets.Layout(width='250px', height='50px')
    )
    boton_volver.on_click(lambda b: mostrar_menu_principal())

    boton_sheets_html = f"""
    <a href="{link_google_sheets}" target="_blank" style="text-decoration:none;">
        <button style="
            background-color:#4CAF50;
            color:white;
            padding:10px 20px;
            border:none;
            border-radius:5px;
            min-width:250px;
            height:50px;
            font-size:16px;
        ">📄 VER RESULTADOS EN SHEETS</button>
    </a>
    """
    display(HTML(f"<br>{boton_sheets_html}<br><br>"))
    display(boton_volver)
# --- PASO 14: Curvas ROC multiclase completas
def ver_curva_roc():
    clear_output()
    print("📉 CURVAS ROC MULTICLASE")

    def plot_roc(y_true, y_pred_proba, titulo):
        y_bin = label_binarize(y_true, classes=np.arange(len(clases)))
        fpr, tpr, roc_auc = {}, {}, {}

        for i in range(len(clases)):
            fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], y_pred_proba[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        plt.figure(figsize=(10, 6))
        for i in range(len(clases)):
            plt.plot(fpr[i], tpr[i], lw=2, label=f"{clases[i]} (AUC = {roc_auc[i]:.2f})")
        plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
        plt.title(titulo)
        plt.xlabel("Falsos Positivos")
        plt.ylabel("Verdaderos Positivos")
        plt.grid(True)
        plt.legend(loc="lower right")
        plt.tight_layout()
        plt.show()

    # Predicciones con probabilidades para test
    proba_interes = modelo_interes.predict(X_test)
    proba_aptitud = modelo_aptitud.predict(X_test)

    plot_roc(y_interes_test, proba_interes, "Curva ROC por Clase - INTERÉS")
    plot_roc(y_aptitud_test, proba_aptitud, "Curva ROC por Clase - APTITUD")

    mostrar_botones_finales()
# --- PASO 15: Exploratory Data Analysis (EDA)
import missingno as msno
from IPython.display import display, HTML

def mostrar_eda():
    clear_output()
    print("📊 Exploratory Data Analysis (EDA)\n")

    # 🔒 Reparar tipos de datos por si fueron alterados
    for col in df.columns[3:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df.iloc[:, 3:] = df.iloc[:, 3:].fillna(0).astype(int)

    # 0. Tipos de datos y valores únicos
    print("🔠 Tipos de datos y valores únicos:")
    tipos = df.dtypes.value_counts()
    plt.figure(figsize=(6, 4))
    tipos.plot(kind='bar', color='coral')
    plt.title("Cantidad de columnas por tipo de dato")
    plt.ylabel("Cantidad")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.grid(axis='y')
    plt.show()
    display(HTML("<br><hr><br>"))

    # 0. Top 10 preguntas menos afirmadas (más informativo que valores únicos)
    print("🔢 Top 10 preguntas con menos afirmaciones (valor 1):")
    conteo_1 = df.iloc[:, 3:].sum().sort_values()
    top10_menos_afirmadas = conteo_1.head(10)

    plt.figure(figsize=(12, 4))
    top10_menos_afirmadas.plot(kind='bar', color='teal')
    plt.title("Top 10 preguntas menos afirmadas (valor 1)")
    plt.ylabel("Cantidad de afirmaciones")
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()
    display(HTML("<br><hr><br>"))

    # 1. Valores faltantes
    print("⚠ Datos faltantes:")
    plt.figure(figsize=(8, 6))
    sns.heatmap(df.isnull(), cbar=False, cmap="YlOrRd")
    plt.title("Mapa de calor de datos faltantes")
    plt.tight_layout()
    plt.show()
    display(HTML("<br><hr><br>"))

    # 2. Distribución por área (boxplots)
    def resumen_por_area(df, mapeo):
        resumen = {}
        for letra, preguntas in mapeo.items():
            columnas = [col for col in df.columns if any(col.startswith(f"{p}.") for p in preguntas)]
            resumen[letra] = df[columnas].sum(axis=1)
        return pd.DataFrame(resumen)

    df_int = resumen_por_area(df, mapeo_interes)
    df_apt = resumen_por_area(df, mapeo_aptitud)

    print("📈 Distribución de puntajes por área (Interés y Aptitud):")
    plt.figure(figsize=(10, 4))
    df_int.plot(kind='box', title='Distribución por área - Interés')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 4))
    df_apt.plot(kind='box', title='Distribución por área - Aptitud')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    display(HTML("<br><hr><br>"))

    # 3. Top 10 preguntas más afirmativas (gráfico tipo punto horizontal)
    print("📊 Top 10 preguntas más afirmativas (formato puntos):")
    top10 = df.iloc[:, 3:].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10, 6))
    plt.scatter(top10.values, top10.index, color='deepskyblue', s=100)
    plt.title("Top 10 preguntas más afirmativas")
    plt.xlabel("Cantidad de respuestas afirmativas")
    plt.grid(axis='x')
    plt.tight_layout()
    plt.show()
    display(HTML("<br><hr><br>"))

    # 4. Correlación entre preguntas seleccionadas
    print("🔗 Matriz de correlación entre preguntas seleccionadas:")
    preguntas_seleccionadas = []
    for lista in mapeo_interes.values():
        preguntas_seleccionadas.extend(lista[:2])
    columnas_heatmap = [col for col in df.columns if any(col.startswith(f"{p}.") for p in preguntas_seleccionadas)]

    plt.figure(figsize=(10, 6))
    sns.heatmap(df[columnas_heatmap].corr(), cmap='coolwarm', annot=True)
    plt.title("Matriz de correlación entre preguntas seleccionadas")
    plt.tight_layout()
    plt.show()
    display(HTML("<br><hr><br>"))

    # 5. Detección de outliers (IQR por área de interés)
    print("📉 Detección de valores atípicos (outliers) por área de interés:")
    outliers_por_area = {}
    for area in df_int.columns:
        Q1 = df_int[area].quantile(0.25)
        Q3 = df_int[area].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df_int[(df_int[area] < Q1 - 1.5 * IQR) | (df_int[area] > Q3 + 1.5 * IQR)]
        outliers_por_area[area] = len(outliers)
    pd.Series(outliers_por_area).plot(kind='bar', color='tomato', title='Cantidad de outliers por área')
    plt.ylabel("Cantidad de outliers")
    plt.tight_layout()
    plt.grid(axis='y')
    plt.show()
    display(HTML("<br><hr><br>"))

    # 6. Calidad de datos: duplicados y constantes
    print("✅ Revisión de calidad de datos:")
    duplicadas = df.duplicated().sum()
    print(f"🔁 Filas duplicadas: {duplicadas}")
    constantes = [col for col in df.columns if df[col].nunique() == 1]
    print(f"⚠ Columnas con valores constantes: {constantes if constantes else 'Ninguna'}")
    display(HTML("<br><hr><br>"))

    # 7. Distribución por área de interés dominante (estilo línea + etiquetas rotadas)
    print("🧠 Distribución de estudiantes por área de interés dominante:")
    areas_interes = []
    for _, alumno in df.iterrows():
        punt_int = calcular_puntajes(alumno, mapeo_interes)
        letra_int = max(punt_int, key=punt_int.get)
        areas_interes.append(area_completa[letra_int])

    df['Área Interés'] = areas_interes
    conteo_area = df['Área Interés'].value_counts().sort_index()

    plt.figure(figsize=(12, 5))
    plt.plot(conteo_area.index, conteo_area.values, marker='o', linestyle='-', color='orchid')
    plt.title("Distribución de estudiantes por área de interés")
    plt.ylabel("Número de estudiantes")
    plt.xlabel("Área de interés")
    plt.xticks(rotation=45, ha='right')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    mostrar_botones_finales()

# --- PASO 16: Menú visual e interacción

dropdown_alumnos = widgets.Dropdown(
    options=df['Nombre y Apellidos'].unique().tolist(),
    description='Alumno:'
)

boton_evaluar = widgets.Button(
    description="✅ EVALUAR SELECCIONADO",
    layout=widgets.Layout(min_width='250px', height='50px'),
    style=dict(button_color='#20B2AA')  # Azul verdoso suave
)

# Botones del menú principal
boton_todos = widgets.Button(
    description='📊 EVALUAR A TODOS',
    layout=widgets.Layout(min_width='250px', height='50px'),
    style=dict(button_color='#007ACC')  # Azul
)
boton_uno = widgets.Button(
    description='🧑‍🎓 EVALUAR UNO',
    layout=widgets.Layout(min_width='250px', height='50px'),
    style=dict(button_color='#28A745')  # Verde
)
boton_reportes = widgets.Button(
    description='📈 REPORTE MÉTRICAS',
    layout=widgets.Layout(min_width='250px', height='50px'),
    style=dict(button_color='#6F42C1')  # Morado
)
boton_eda = widgets.Button(
    description='📊 ANÁLISIS EDA',
    layout=widgets.Layout(min_width='250px', height='50px'),
    style=dict(button_color='#FF5733')  # Naranja
)
boton_roc = widgets.Button(
    description='📉 CURVAS ROC',
    layout=widgets.Layout(min_width='250px', height='50px'),
    style=dict(button_color='#17A2B8')  # Celeste
)


# Limpiar eventos antiguos para evitar duplicación
for b in [boton_todos, boton_uno, boton_reportes, boton_eda, boton_roc, boton_evaluar]:
    try: b._click_handlers.callbacks.clear()
    except: pass

# Enlaces de eventos
boton_todos.on_click(lambda b: evaluar_todos())
boton_uno.on_click(lambda b: [clear_output(), mostrar_menu_principal(), display(dropdown_alumnos), display(boton_evaluar)])
boton_reportes.on_click(lambda b: ver_reportes())
boton_eda.on_click(lambda b: mostrar_eda())
boton_roc.on_click(lambda b: ver_curva_roc())
boton_evaluar.on_click(lambda b: evaluar_uno(dropdown_alumnos.value))

# Mostrar menú inicial
def mostrar_menu_principal():
    clear_output()

    display(HTML("""
    <div style="text-align:center; padding: 20px;">
        <h2 style="color:#00FFFF; font-size:24px;">SISTEMA DE TEST VOCACIONAL INTELIGENTE</h2>
        <h1 style="color:#DA70D6; font-size:28px;">MENÚ PRINCIPAL</h1>
    </div>
    """))

    # Contenedor horizontal con espaciado entre botones
    contenedor = widgets.HBox(
        [boton_todos, boton_uno, boton_reportes, boton_eda, boton_roc],
        layout=widgets.Layout(
            display='flex',
            flex_flow='row wrap',
            justify_content='center',
            align_items='center',
            gap='16px'  # Espacio entre botones
        )
    )

    display(contenedor)

# Llamada inicial para mostrar el menú al arrancar
mostrar_menu_principal()
